# Process and Upload Data to Database

#### Updated: Oct 7, 2022

#  

Streamline the uploading of data to AWS RDS MySQL database:

In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
import time
import os

In [2]:
dataDir = '/Users/rnaidoo/Documents/Canada-Secure/GAC/2022_MDRID/Projects_data/OpenAlex/'

In [3]:
server_name = 'mdrid-1.cfgquvzxu6rh.ca-central-1.rds.amazonaws.com'
database_name = 'MDRID_1'
username_ = 'admin'
pwd = '7jLkMAyKgESupoJiueft'

connection_url = sa.engine.URL.create(
    "mysql+pymysql",
    host=server_name,
    username=username_,
    password=pwd,
    database=database_name
)
engine = create_engine(connection_url)

#  

#### Functions:

In [4]:
def SQL_cmd(sql_cmd, engine, printRows=False):
    with engine.connect() as connection:
        result = connection.execute(sql_cmd)
        rows = []
        if printRows:
            for row in result:
                rows.append(str(row)[2:-3])
    
    return rows

In [5]:
def get_dtypes(df_in):

    dtype_dict = {}
    for i in range(0, len(df_in.columns)):
        #Get max character length for column:
        ns = []
        nset = 0
        for j in range(0, len(df_in)):
            n = len(str(df_in[df_in.columns[i]].iloc[j]))
            ns.append(n)
        nmax = max(ns)
        if nmax < 50:
            nset = 50
        #elif nmax > 4000:
        #    nset = 'max'
        else:
            nset = nmax

        dtype_dict.update({df_in.columns[i]: sa.types.String(length=nset)})

    return dtype_dict

#  

In [6]:
#ISO-3166 2-letter country codes

country_code = 'PT'
country_code_collab = 'CA'

dataDir_first_auth = dataDir + 'works_' + country_code + '_first_auth/'
dataDir_col = dataDir_first_auth + country_code_collab + '_col/'

os.makedirs(dataDir_col, exist_ok=True)

#  

#### Works table - process and upload:

In [7]:
pubs_fn = 'works_' + country_code + '_first_auth_' + country_code_collab + '_col_since2017'
pubs_fn_app = pubs_fn + '_APP'

In [8]:
rows = SQL_cmd('SHOW TABLES', engine=engine, printRows=True)
if pubs_fn_app in rows:
    print("'" + pubs_fn_app + "' exists!")

else:
    df_pubs_first_auth = pd.read_csv(dataDir_col + pubs_fn + '.csv')
    print(len(df_pubs_first_auth))
    df_pubs_first_auth2 = df_pubs_first_auth[['id', 'title', 'publication_date', 'authorships', 'cited_by_count', 'first_author', 'first_auth_orcid', 'first_auth_inst', 'first_auth_country', 'collaborators_of_interest']]
    df_pubs_first_auth2.to_csv(dataDir_col + pubs_fn_app + '.csv', index=False)
    dtype_dict = get_dtypes(df_pubs_first_auth2)

    df_load = pd.read_csv(dataDir_col + pubs_fn_app + '.csv')

    start = time.time()
    
    df_load.to_sql(name=pubs_fn_app, con=engine, if_exists='replace', index=False, dtype=dtype_dict)
    #Set primary key
    sql = 'ALTER TABLE ' + pubs_fn_app + ' ADD PRIMARY KEY (id);'
    SQL_cmd(sql, engine=engine, printRows=False)
    
    end = time.time()
    t = end - start
    rate = t/len(df_load)
    print('Upload time: ' + str(round(t/60, 2)) + ' min (' + str(round(rate, 3)) + ' s/row)')

    df_loaded = pd.read_sql_query('SELECT * FROM ' + pubs_fn_app + ' LIMIT 5', engine)
    df_loaded

FileNotFoundError: [Errno 2] No such file or directory: '/Users/rnaidoo/Documents/Canada-Secure/GAC/2022_MDRID/Projects_data/OpenAlex/works_PT_first_auth/CA_col/works_PT_first_auth_CA_col_since2017.csv'

In [ ]:
'''start = time.time()
df_loaded = pd.read_sql_query('SELECT * FROM ' + pubs_fn_app, engine)
end = time.time()
print(len(df_loaded))
t = end - start
rate = t/len(df_loaded)
print('Download time: ' + str(round(t/60, 2)) + ' min (' + str(round(rate, 3)) + ' s/row)')
df_loaded.head()'''

#  

#### Concepts table - process and upload:

In [ ]:
conc_fn = 'concepts_by_pub_' + country_code + '_first_auth_since2017'
conc_fn_app = conc_fn + '_APP'

In [ ]:
rows = SQL_cmd('SHOW TABLES', engine=engine, printRows=True)
if conc_fn_app in rows:
    print("'" + conc_fn_app + "' exists!")

else:
    df_conc_first_auth = pd.read_csv(dataDir_first_auth + conc_fn + '.csv')
    print(len(df_conc_first_auth))
    df_conc_first_auth2 = df_conc_first_auth[['pub_id', 'concept', 'concept_level']]
    df_conc_first_auth2.to_csv(dataDir_first_auth + conc_fn_app + '.csv', index=False)
    dtype_dict = get_dtypes(df_conc_first_auth2)

    df_load = pd.read_csv(dataDir_first_auth + conc_fn_app + '.csv')

    start = time.time()
    
    df_load.to_sql(name=conc_fn_app, con=engine, if_exists='replace', index=False, dtype=dtype_dict)
    #Set primary key
    sql = 'ALTER TABLE ' + conc_fn_app + ' ADD PRIMARY KEY (pub_id);'
    SQL_cmd(sql, engine=engine, printRows=False)
    
    end = time.time()
    t = end - start
    rate = t/len(df_load)
    print('Upload time: ' + str(round(t/60, 2)) + ' min (' + str(round(rate, 3)) + ' s/row)')

    df_loaded = pd.read_sql_query('SELECT * FROM ' + conc_fn_app + ' LIMIT 5', engine)
    df_loaded

#  

#### Show tables in database:

In [ ]:
SQL_cmd('SHOW TABLES', engine=engine, printRows=True)